In [0]:
dataset_storage_account_name="melikadatabricksstorage"
dataset_container_name="geospatial-dataset"
dataset_dir="vector/uk"

catalog_name = "geospatial"

In [0]:
spark.conf.set(f"spark.hadoop.fs.azure.account.auth.type.{dataset_storage_account_name}.dfs.core.windows.net", "ManagedIdentity")
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.id.{dataset_storage_account_name}.dfs.core.windows.net","a6d7d9b6-d9a1-4711-a74b-3cdfc43f3dd8")

In [0]:
from sedona.spark import *

config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.3_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5'). \
    getOrCreate()

In [0]:
from sedona.spark import *

sedona = SedonaContext.create(config)

In [0]:
schema_tables = {
    "lookups": {
        "bdline_gb.gpkg": ["boundary_line_ceremonial_counties"],
        "opname_gb.gpkg": ["named_place"]
    },
    "greenspaces": {
        "opgrsp_gb.gpkg": ["greenspace_site", "access_point"]
    },
    "networks": {
        "oproad_gb.gpkg": ["road_link", "road_node"],
        "oprvrs_gb.gpkg": ["watercourse_link", "hydro_node"]
    },
    "heights": {
        "terr50_gb.gpkg": ["contour_line"]
    }
}


for schema, files in schema_tables.items():
    for gpkg_file, layers in files.items():
        for table_name in layers:
            df = sedona.read.format("geopackage").option("tableName", table_name).load(f"abfss://{dataset_container_name}@{dataset_storage_account_name}.dfs.core.windows.net/{dataset_dir}/{gpkg_file}")
            df.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema}.{table_name}")


